# cosmodesi+desilike tutorial


## Perspectiva general
Este tutorial tiene una estructura general similar al tutorial presentado por Aranud en el DESI Summer Collaboration Meeting 2023 
(https://github.com/cosmodesi/cosmodesi-tutorials). También está inspirado en algunos de los notebooks de ejemplo, y en el tutorial de desilike (https://github.com/cosmodesi/desilike-tutorial).

El objetivo es dar una introducción a la sintaxis del código, mostrar un ejemplo de cómo correr el pipeline para hacer un fit de clustering de principio a fin y dar a los asistentes las noción general de la paquetería. Se asume que los participantes tienen nociones elementales de los diferentes elementos que conforman el pipeline y la física detrás (códigos de Boltzmann, función de correlación, espectro de potencias, BAO, full shape fit, etc.)

El notebook está pensado para trabajarse de la siguiente forma: Cada parte de la paquetería se introduce de forma breve, se muestran algunas característcas y funcionalidades. Posteriormente sigue un ejercicio pensando para resolverse en conjunto con los participantes de forma guiada.

## Contenido
- **cosmoprimo**: Cosmología primordial
- **pypower**: Computo de espectro de potencias
- **pycorr**: Computo de fución de correlación.
- **desilike**: Templates, teorías, observables, likelihoods y fits.


#### Recordatorio antes de comenzar..
Tomado del tutorial de Arnaud: *None of this is part of the "DESI project", i.e. everything relies on volunteering / service work.*


## cosmoprimo
- Documentación: https://cosmoprimo.readthedocs.io/en/latest/index.html
- Notebook de ejemplo: https://github.com/cosmodesi/cosmoprimo/blob/main/nb/examples.ipynb

Cosmología primordial. Trabaja con camb/CLASS de fondo, pero con un "framework" consistente.

In [ ]:
from cosmoprimo import Cosmology, fiducial

# DESI fiducial cosmology
cosmo = fiducial.DESI()
type(cosmo)

In [ ]:
# explore some features
help(cosmo)

In [ ]:
# print parameter dictionary
print(cosmo.get_default_parameters())
print('\n')
print(cosmo.get_default_parameters(include_conflicts=False))

In [ ]:
# Use camb or class(default) as the engine for cosmological calculations
#(Background, Thermodynamics, Primordial, Perturbations, Transfer, Harmonic, Fourier)
cosmo_camb = Cosmology(engine='camb', **cosmo.params.copy())
ba = cosmo_camb.get_background()

In [ ]:
# Calculate background products
ba.angular_diameter_distance(0.1)

In [ ]:
# engine can be changed "in-place"
print(cosmo.get_background(engine='class').angular_diameter_distance(0.1))
print(cosmo.get_background(engine='camb').angular_diameter_distance(0.1))

In [ ]:
# One can use some shortcuts directly
unCl = cosmo_camb.unlensed_cl()

import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
ell, tt = unCl['ell'], unCl['tt']
ax.plot(ell, ell * (ell + 1) / (2 * np.pi) * tt)
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\ell(\ell+1)C_{\ell}/(2\pi)$')

In [ ]:
# Instead of using the shorcuts, one can use one of the underlying classes
from cosmoprimo import Background, Thermodynamics, Primordial, Transfer, Harmonic, Fourier

In [ ]:
fo = Fourier(cosmo, engine='class')
z = 0.0
pk = fo.pk_interpolator().to_1d(z=z)
pk_cross = fo.pk_interpolator(of=('delta_cb', 'theta_cb')).to_1d(z=z)
k = np.logspace(-3, 2, 1000)
fig, ax = plt.subplots()
ax.plot(k, pk(k), label=r'$P_m$')
ax.plot(k, pk_cross(k), label=r'$P_{\delta_{cb}\theta_{cb}}$')


ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$k$ [$h/\mathrm{Mpc}$]')
ax.set_ylabel(r'$P(k)$ [$(\mathrm{Mpc}/h)^{3}$]')
ax.legend()

#### Ejercicio
Create a cosmology instance with the same parameters as the Planck2018 cosmology but with 20% more cold dark matter. Plot the luminous distance for both cosmologies as a function of z.

## pypower
- Documentación: https://pypower.readthedocs.io/en/latest/
- Notebooks de ejemplo: https://github.com/cosmodesi/pypower/tree/main/nb


Computo de espectro de potencias. Algunas reimplentaciones basadas en nbodykit pero con muchas más funcionalidades.

In [ ]:
# First let us load a mock galaxy catalog
from mockfactory import Catalog
import fitsio
import numpy as np

In [ ]:
# This catalog corresponds to a cubic box of side lenght = 1000 Mpc/h
# positions are x, y, z coordinates, z coordinate includes RSD
data = fitsio.read('mock_catalog_bias2.0_z0.8.fits')
data['Position']

In [ ]:
print(np.min(data['Position'][:,0]), np.max(data['Position'][:,0]))
print(np.min(data['Position'][:,1]), np.max(data['Position'][:,1]))
print(np.min(data['Position'][:,2]), np.max(data['Position'][:,2]))

In [ ]:
# wrap z coordinate around
data['Position'][:, 2] = data['Position'][:, 2] % 1000.

In [ ]:
from pypower import CatalogFFTPower, PowerSpectrumMultipoles, setup_logging
setup_logging()

In [ ]:
# Most of the time it's easier to use the high-level interface
power = CatalogFFTPower(data_positions1=data['Position'],
                        boxsize=1000., boxcenter=500.,
                        interlacing = 2, los='z', resampler='tsc',
                        edges={'step': 0.001}, ells=(0, 2),
                        position_type='pos', nmesh=256)

In [ ]:
type(power)

In [ ]:
poles = power.poles
type(poles)

In [ ]:
poles.plot()

In [ ]:
# Check attributes
poles.attrs

In [ ]:
# Rebinning
print('Initial k-modes:\n', poles.kavg)
#First truncate
poles.slice(slice(0, 500))
#Then rebin
poles.rebin(5)
print('After rebinning:\n', poles.kavg)

In [ ]:
poles.plot()

In [ ]:
# Save your results
poles.save('pk_multipoles')
# As a txt file
poles.save_txt('pk_multipoles.txt', complex=False)

In [ ]:
# load npy file
poles = PowerSpectrumMultipoles.load('pk_multipoles.npy')

#### Ejercicio
Calcular primero un `CatalogMesh` y calcular el power spectrum a partir de este (como en nbodykit). Utilizar `resampler=ngp` y comparar con el resultado previo.

## pycorr
- Documentación: https://py2pcf.readthedocs.io/en/latest/index.html
- Notebooks de ejemplo: https://github.com/cosmodesi/pycorr/tree/main/nb

Computo de función de correlación. El engine de fondo es `corrfunc`.

In [ ]:
from pycorr import TwoPointCorrelationFunction

# high-level interface similar to pypower
# for a periodic box we can simply use the natural estimator
result = TwoPointCorrelationFunction(mode='smu', data_positions1=data['Position'],
                                     boxsize=1000., boxcenter=500., los='z',
                                     edges=(np.linspace(0., 200., 201), np.linspace(-1., 1., 201)),
                                     position_type='pos', dtype='f8')

In [ ]:
result.plot(mode='poles')

# desilike
- Documentación: https://desilike.readthedocs.io/en/latest/index.html
- Notebooks de ejemplo: https://github.com/cosmodesi/desilike/tree/main/nb

Provee un 'framework' consistente para el pipeline de DESI: *teorías, emuladores, observables, likelihoods, samplers, profilers*.

In [ ]:
from desilike import setup_logging
setup_logging()

### Theories

In [ ]:
from desilike.theories.galaxy_clustering import (DirectPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles,
                                                 FOLPSTracerPowerSpectrumMultipoles, DampedBAOWigglesTracerPowerSpectrumMultipoles,
                                                 StandardPowerSpectrumTemplate)

In [ ]:
# Direct power spectrum template
template = DirectPowerSpectrumTemplate(z=0.8, fiducial='DESI')
theory = FOLPSTracerPowerSpectrumMultipoles(template=template)

In [ ]:
# Template and theory parameters
print(template.params)
print(theory.params)
print(theory.all_params)

In [ ]:
# Standard compression
template = StandardPowerSpectrumTemplate(z=0.8, fiducial='DESI')
print(template.params)
theory = FOLPSTracerPowerSpectrumMultipoles(template=template)
print(theory.params)

In [ ]:
# just for fun...
# how does the AP parameters change the clustering
cmap = plt.get_cmap('jet', 5)
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
values = np.linspace(0.98, 1.2, 5)
for j, param in enumerate(['qpar', 'qper']):
    for ivalue, value in enumerate(values):
        pk_poles = theory(**{param:value})
        for ill, ell in enumerate(theory.ells):
            label = value if ill==0 else None
            axes[j].plot(theory.k, theory.k * pk_poles[ill], color=cmap(ivalue), label=label)
            axes[j].legend()
axes[0].set_title(r'$q_\parallel$')
axes[1].set_title(r'$q_\perp$')

In [ ]:
# For the following fits we will use the Kaiser power spectrum as the model
# To ensure computation times are reasonable
template = DirectPowerSpectrumTemplate(z=0.8, fiducial='DESI')
for param in ['omega_b', 'n_s']: template.params[param].update(fixed=True)
theory = KaiserTracerPowerSpectrumMultipoles(template=template)
theory.params['b1'].update(value=2.)

### Observable and Likelihood

In [ ]:
# first load covariance matrix
covariance = np.load('covariance_pk.npy')

In [ ]:
# Arnaud's function to cut covariance
def cut_matrix(cov, xcov, ellscov, xlim):
    import numpy as np
    assert len(cov) == len(xcov) * len(ellscov), 'Input matrix has size {}, different than {} x {}'.format(len(cov), len(xcov), len(ellscov))
    indices = []
    for ell, xlim in xlim.items():
        index = ellscov.index(ell) * len(xcov) + np.arange(len(xcov))
        index = index[(xcov >= xlim[0]) & (xcov <= xlim[1])]
        indices.append(index)
    indices = np.concatenate(indices, axis=0)
    return cov[np.ix_(indices, indices)]

In [ ]:
klim={0: [0.02, 0.2], 2: [0.02, 0.2]} # dictionary of ranges
k = 1/2 * (poles.kedges[1:] + poles.kedges[:-1])
covariance = cut_matrix(covariance, k, (0, 2, 4), klim)

In [ ]:
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood
# Define observable
# one could also use the correlation funcition multipoles
observable = TracerPowerSpectrumMultipolesObservable(data=poles, covariance=covariance,
                                                     klim=klim, 
                                                     theory=theory)

likelihood = ObservablesGaussianLikelihood(observables=[observable])
#itinialise the likelihood
likelihood()

### Emulators

In [ ]:
from desilike.emulators import EmulatedCalculator, Emulator, TaylorEmulatorEngine

In [ ]:
# Use Taylor expansion emulator (with finate differences)
# Alternativaly use mlp emulator
emulator = Emulator(theory, engine=TaylorEmulatorEngine(order={'*': 2, 'sn0': 1}))  # order 2 except for sn0 
emulator.set_samples()
emulator.fit()

In [ ]:
emulator.save('emulator_kaiser')

In [ ]:
# update observable to use the emulator instead of using theory directly
observable.init.update(theory=emulator.to_calculator())

### Profilers

In [ ]:
from desilike.profilers import MinuitProfiler

# Cern's Minuit optimiser
profiler = MinuitProfiler(likelihood, seed=42)

#print('something went wrong!')
profiles = profiler.maximize(niterations=1)

In [ ]:
# print statistics
print(profiler.profiles.to_stats(tablefmt='pretty'))

In [ ]:
# Evaluate likelihood at bestfit params and plot observable
print(likelihood(**profiler.profiles.bestfit.choice(params=likelihood.varied_params)))
observable.plot()

### Samplers

In [ ]:
# Now let us run an MCMC

# First let us reduced the parameter space by fixing some params
# to the values we obtained above
likelihood.all_params.update(name='logA', value=2.72, fixed=True)
likelihood.all_params.update(name='b1', value=2.3, fixed=True)
likelihood.all_params.update(name='sn0', value=0.094, fixed=True)

In [ ]:
likelihood.varied_params.basenames

In [ ]:
# Let us use the Emcee Sampler
from desilike.samplers import EmceeSampler

sampler = EmceeSampler(likelihood, save_fn='chain_fs_kaiser_*.npy', seed=42)
sampler.run(max_iterations=3000, check={'max_eigen_gr': 0.03})

In [ ]:
chain = sampler.chains[0].remove_burnin(0.3)
print(chain.to_stats(tablefmt='pretty'))

In [ ]:
from desilike.samples import plotting
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import getdist
import IPython
plotting.plot_triangle(chain, markers={'Omega_m': cosmo.Omega0_m, 'h': cosmo.h});

#### Ejercicio
Correr un fit standard de BAO utilizando (o alguna otra sugerencia) el `MinuitProfiler` con los mismos datos anteriores.